# ooookay you motherfuckers!
lets roll!


!pip install pyspark


from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .appName("xyi") \
    .master("local") \
    .getOrCreate()

In [2]:
import pyarrow.parquet as pq
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import LabelEncoder
import random
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier

In [3]:
def open_df(part):
    
    import pandas as pd
    
    # open
    df = pd.read_csv('train_data/tst_'+str(part)+'.csv', header=None)
    
    # очень неудобные названия
    col_names = dict()
    for i in range(len(df.columns)):
                       col_names[df.columns[i]] = cols[i]
    df = df.rename(columns=col_names)
    return df

In [4]:
def sample_df(df3, total_rows =  100000, neg_percent = 50, pos_percent = 50):
    print( 'sample_df3 start')
    df3_pos = df3[df3['flag'] == 1].sample(int(total_rows / 100 * pos_percent))
    df3_neg = df3[df3['flag'] == 0].sample(int(total_rows / 100 * neg_percent))
    df3_pos = df3_pos.reset_index()
    df3_neg = df3_neg.reset_index()
    df3_pos = df3_pos.drop('index', axis=1)
    df3_neg = df3_neg.drop('index', axis=1)
    df3 = pd.concat([df3_pos, df3_neg])
    
    print( 'sample_df3 end')
    print('-')      
    #print('-')      
    #print('-') 
    
    return df3

In [5]:

cols = ['id',
 'rn',
 'pre_since_opened',
 'pre_since_confirmed',
 'pre_pterm',
 'pre_fterm',
 'pre_till_pclose',
 'pre_till_fclose',
 'pre_loans_credit_limit',
 'pre_loans_next_pay_summ',
 'pre_loans_outstanding',
 'pre_loans_total_overdue',
 'pre_loans_max_overdue_sum',
 'pre_loans_credit_cost_rate',
 'pre_loans5',
 'pre_loans530',
 'pre_loans3060',
 'pre_loans6090',
 'pre_loans90',
 'is_zero_loans5',
 'is_zero_loans530',
 'is_zero_loans3060',
 'is_zero_loans6090',
 'is_zero_loans90',
 'pre_util',
 'pre_over2limit',
 'pre_maxover2limit',
 'is_zero_util',
 'is_zero_over2limit',
 'is_zero_maxover2limit',
 'enc_paym_0',
 'enc_paym_1',
 'enc_paym_2',
 'enc_paym_3',
 'enc_paym_4',
 'enc_paym_5',
 'enc_paym_6',
 'enc_paym_7',
 'enc_paym_8',
 'enc_paym_9',
 'enc_paym_10',
 'enc_paym_11',
 'enc_paym_12',
 'enc_paym_13',
 'enc_paym_14',
 'enc_paym_15',
 'enc_paym_16',
 'enc_paym_17',
 'enc_paym_18',
 'enc_paym_19',
 'enc_paym_20',
 'enc_paym_21',
 'enc_paym_22',
 'enc_paym_23',
 'enc_paym_24',
 'enc_loans_account_holder_type',
 'enc_loans_credit_status',
 'enc_loans_credit_type',
 'enc_loans_account_cur',
 'pclose_flag',
 'fclose_flag']

In [6]:
df = open_df(0)
# df2 = open_df(1)
# df3 = open_df(2)

In [7]:
answers = pd.read_csv('train_target.csv')

In [8]:
answers.flag.value_counts()[1] / answers.flag.value_counts()[0]

0.03678585326438938

In [10]:
answers.flag.value_counts()

0    2893558
1     106442
Name: flag, dtype: int64

In [11]:
answers_dict = dict()

In [12]:
answers.columns

Index(['id', 'flag'], dtype='object')

In [13]:
df = pd.merge(df, answers, on='id', how='left')


In [14]:
df_balanced = sample_df(df, 100000, 50, 50)

sample_df3 start
sample_df3 end
-


sns.heatmap(df_balanced.corr(), cmap='coolwarm', linewidths=0.5)


In [15]:
df_corr = df_balanced.corr()

In [16]:
columns = df_corr.columns
rows = df_corr.columns

In [17]:
corrs = []
for col in columns:
    corrs.append((df_balanced.flag.corr(df_balanced[col]), col))

In [18]:
corrs = corrs[:-1]

In [ ]:
for entry in corrs:
    

In [41]:
df_corr.loc['enc_paym_22', 'enc_paym_23']

0.9565579853950436

In [13]:
id_list = df.id.unique()

In [14]:
check = df[['flag', 'id']]

In [15]:
check[check.id == 4 ]#.iloc[0,0]

,flag,id
42,0,4


positive_ids = []
check = df[['flag', 'id']]
for entry in id_list:
    print(entry)
    if check[check.id == entry ].iloc[0,0] == 1:
        positive_ids.append(1)
positive_ids
    

df2.columns

In [18]:
sum(positive_ids) / len(id_list)

0.030936

In [28]:
df_train, df_test = train_test_split(df_balanced, stratify=df_balanced['flag'], test_size=0.3)

In [29]:
x_train = df_train.drop('flag', axis=1)
y_train = df_train.flag
x_test = df_test.drop('flag', axis=1)
y_test = df_test.flag

rf =  RandomForestClassifier()

rf.fit(x_train, y_train)

In [49]:
model = LogisticRegression()

In [21]:
model = RandomForestClassifier()

In [59]:
model  = MLPClassifier()

In [22]:
model.fit(x_train, y_train)

RandomForestClassifier()

In [23]:
pred_train = model.predict(x_train)
pred_test = model.predict(x_test)
print('train score acc - ', accuracy_score(y_train, pred_train))
print('test score acc - ', accuracy_score(y_test, pred_test))

prob_train = model.predict_proba(x_train)[:, 1]
prob_test = model.predict_proba(x_test)[:, 1]
print('train score roc - ', roc_auc_score(y_train, prob_train))
print('test score roc - ', roc_auc_score(y_test, prob_test))

train score acc -  1.0
test score acc -  0.6074
train score roc -  1.0
test score roc -  0.6484391444444444


In [24]:
# неправдоподобно высокие показатели, попробуем другую метрику

In [25]:
df2 = open_df(1)
df2 = pd.merge(df2, answers, on='id', how='left')


In [26]:
x_train = df2.drop('flag', axis=1)
y_train = df2.flag


In [27]:
pred_train = model.predict(x_train)
print('train score acc - ', accuracy_score(y_train, pred_train))

prob_train = model.predict_proba(x_train)[:, 1]
print('train score roc - ', roc_auc_score(y_train, prob_train))


train score acc -  0.6884236926750582
train score roc -  0.6235708113707962


In [30]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

model = RandomForestClassifier()

# Perform Grid Search using Random Forest Classifier
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='roc_auc')
grid_search.fit(x_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}
Best Score: 0.6586614683736624


In [32]:
lr.predict_proba(x_train)

array([[0.49930566, 0.50069434],
       [0.49827974, 0.50172026],
       [0.49695717, 0.50304283],
       ...,
       [0.49686093, 0.50313907],
       [0.49893878, 0.50106122],
       [0.49670406, 0.50329594]])

In [27]:
df2 = pd.read_csv('train_data/tst_1.csv', header=None)

In [28]:
df2.shape

(2107305, 61)

In [29]:
# очень неудобные названия
col_names = dict()
for i in range(len(df2.columns)):
                   col_names[df2.columns[i]] = cols[i]
df2 = df2.rename(columns=col_names)

In [30]:
df2['flag'] = df2.id.apply(lambda x:answers_dict[x])


KeyError: 250000

In [ ]:
df2_train, df2_test = train_test_split(df2, stratify=df2['flag'], test_size=0.3)
x2_train = df2_train.drop('flag', axis=1)
y2_train = df2_train.flag
x2_test = df2_test.drop('flag', axis=1)
y2_test = df2_test.flag

In [ ]:
pred2_train = lr.predict(x2_train)
pred2_test = lr.predict(x2_test)
print('train score - ', accuracy_score(y2_train, pred2_train))
print('test score - ', accuracy_score(y2_test, pred2_test))

In [ ]:
y2_test.value_counts()

In [ ]:
counter = 0
for entry in pred2_test:
    if entry == 1:
        counter +=1
        
print(counter)

In [75]:
# путь до данных на компьютере
path = 'train_data/'

In [48]:
import os
import pandas as pd
import tqdm


def read_parquet_dataset_from_local(path_to_dataset: str, start_from: int = 0,
                                     num_parts_to_read: int = 2, columns=None, verbose=False) -> pd.DataFrame:
    """
    читает num_parts_to_read партиций, преобразовывает их к pd.DataFrame и возвращает
    :param path_to_dataset: путь до директории с партициями
    :param start_from: номер партиции, с которой нужно начать чтение
    :param num_parts_to_read: количество партиций, которые требуется прочитать
    :param columns: список колонок, которые нужно прочитать из партиции
    :return: pd.DataFrame
    """

    res = []
    dataset_paths = sorted([os.path.join(path_to_dataset, filename) for filename in os.listdir(path_to_dataset)
                              if filename.startswith('train')])
    print(dataset_paths)

    start_from = max(0, start_from)
    chunks = dataset_paths[start_from: start_from + num_parts_to_read]
    if verbose:
        print('Reading chunks:\n')
        for chunk in chunks:
            print(chunk)
    for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):
        print('chunk_path', chunk_path)
        chunk = pd.read_parquet(chunk_path,columns=columns)
        res.append(chunk)

    return pd.concat(res).reset_index(drop=True)

In [50]:
# answers
check = []
for i in range(len(answers)-1):
#     if i == 
    check.append(answers.id[i]+1 == answers.id[i+1])